In [402]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from operator import itemgetter
import seaborn as sns

In [403]:
def add_data_info():
    df1 = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-from-2017.csv', encoding= 'unicode_escape', low_memory=False)
    df2 = pd.read_csv('raw_data/charting-m-matches.csv', encoding= 'unicode_escape', low_memory=False)
    
    #get separated values from m-matches list of games - based on matchid
    df = pd.merge(df1,df2[['match_id','Date','Tournament','Round','Surface','Player 1','Player 2']], on=['match_id'], how='left')
    
    #add the condition of "Grand Slam" - 5 or 3 sets
    conditions = df['Tournament'].isin(["Wimbledon", "Australian Open", "US Open", "Roland Garros"])
    values = [1,0]

    # Create a new column  based on the conditions and values
    df['Grand Slam'] = pd.Series(pd.NA)
    df.loc[conditions, 'Grand Slam'] = values[0]
    df.loc[~conditions, 'Grand Slam'] = values[1]
    
    # save the original dataset with enriched data for future features
    df.to_csv('raw_data/charting-m-points-from-2017-enriched.csv', index=False)

In [404]:
def filter_player(df,opponent):
    
    # Read the dataset into a pandas DataFrame - from enriched file
    df = pd.read_csv(df, encoding= 'unicode_escape', low_memory=False)
    
    ## filtra pelo  oponente
    df = df[(df["Player 1"].str.contains(opponent)) | (df["Player 2"].str.contains(opponent))].copy()
    
    ##define se está sacando e se ganhou o ponto
    df['i_serve'] = np.where(((df['Svr'] == 1) & (df['Player 1'] == opponent )) | ((df['Svr'] == 2) & (df['Player 2'] == opponent)), 0, 1)
    df['i_win'] = np.where(((df['isSvrWinner'] == df['i_serve'])), 1, 0)
    df['is_second_service'] = np.where(((df['2nd'].isnull())), 0, 1)
    
    ## direção do saque
    # primeiro caractere do segundo saque
    df['dir_srv'] = df.loc[:, '2nd'].str[0]
    ## completa com o primeiro caractere do primeiro saque quando ele entrou
    df['dir_srv'].fillna(df.loc[:, '1st'].str[0], inplace=True)

    
    ##reseta o indice
    df.reset_index(inplace=True)
    
    ## salva arquivo e retorna dataset da funcao
    csv_file = "raw_data/" + opponent + "_points.csv"
    df.to_csv(csv_file, index=False)
    return df

In [405]:
meaning_char = pd.read_csv('https://raw.githubusercontent.com/eduardoscovino/tennis-analytics/master/code_map.csv',
                           encoding= 'unicode_escape', sep=';')

In [406]:
add_data_info()

In [407]:
df = filter_player('raw_data/charting-m-points-from-2017-enriched.csv','Novak Djokovic')

In [408]:
## serve direction encode 
df = df[df['dir_srv'].isin(['4','5','6'])]

## to numeric for rallyCount and serve direction
df['rallyCount']=pd.to_numeric(df['rallyCount'])
df['dir_srv']=pd.to_numeric(df['dir_srv'])

In [409]:
#Spliting Serve and Return
df['rally_desc'] = df.loc[:, '2nd']
df['rally_desc'].fillna(df.loc[:, '1st'], inplace=True)

##df['Serve'] = df.loc[:, 'rally_desc'].str[0:1]

df['rally_desc'] = df['rally_desc'].str[1:]

sc = np.array(meaning_char[~meaning_char['code'].isin(['f', 'b', 'r', 's', 'v', 'l', 'o', 'z', 'p', 'u', 'y', 'h', 'i', 'j', 'k','m', '1', '2', '3', '7', '8', '9'])]['code'])

#Spliting every shot
df['rally_desc'] = df['rally_desc'].apply(lambda x: ''.join([c for c in x if c not in sc]))
df['rally_desc'] = df['rally_desc'].str.replace('f',' f').str.replace('b',' b').str.replace('s',' s').str.replace('r',' r').str.replace('v',' v').str.replace('l',' l').str.replace('o',' o').str.replace('z',' z').str.replace('p',' p').str.replace('u',' u').str.replace('y',' y').str.replace('h',' h').str.replace('i',' i').str.replace('j',' j').str.replace('k',' k').str.replace('m',' m').str.replace('!','').str.replace('+','').str.replace(';','').str.replace('^','').str.replace('C','')
df['rally_desc'] = df['rally_desc'].map(lambda x: x.lstrip(' ').rstrip(' '))

df['rally_tratada'] = (df['rallyCount']/2).apply(np.floor).astype('int')

/var/folders/kk/0r0_hyfd673456ly0hptrbsc0000gn/T/ipykernel_51892/3403707337.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rally_desc'] = df['rally_desc'].str.replace('f',' f').str.replace('b',' b').str.replace('s',' s').str.replace('r',' r').str.replace('v',' v').str.replace('l',' l').str.replace('o',' o').str.replace('z',' z').str.replace('p',' p').str.replace('u',' u').str.replace('y',' y').str.replace('h',' h').str.replace('i',' i').str.replace('j',' j').str.replace('k',' k').str.replace('m',' m').str.replace('!','').str.replace('+','').str.replace(';','').str.replace('^','').str.replace('C','')


### Adição da coluna how_ended

In [410]:
# df['end']
df['how_ended'] = df.loc[:, '2nd'].str[-1]
df['how_ended'].fillna(df.loc[:, '1st'].str[-1], inplace=True)

In [411]:
df['isUnforced'] = df['isUnforced'].replace({True: 1, False: 0})
df['isForced'] = df['isForced'].replace({True: 1, False: 0})
df['serve_return'] = df['rally_desc'].map(lambda x: x.split(' ')).str[0]

### DATASET FINAL 

In [412]:
# Data set final sendo "i serve = 0" - receiving
df = df[df['i_serve'] == 1]
df = df[df['serve_return'] != '']
df = df[df['i_win'] == 1]

## Dataset finalizado ##

In [413]:
df.head()

,index,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,TBpt,Svr,Ret,Serving,1st,2nd,Notes,1stSV,2ndSV,1stIn,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,Date,Tournament,Round,Surface,Player 1,Player 2,Grand Slam,i_serve,i_win,is_second_service,dir_srv,rally_desc,rally_tratada,how_ended,serve_return
5,642,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,6,0,0,1,0,0-0,2 (1),1,0,NaN,2,1,CR,4n,6b37b2f3s2f1f1f3w@,NaN,0,0.0,0,1.0,False,False,False,0,1,False,2,1,7,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,1,1,1,6,b37 b2 f3 s2 f1 f1 f3,3,@,b37
9,646,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,10,0,0,1,0,15-40,2 (5),1,0,NaN,2,1,CR,6d,5b3w@,NaN,0,0.0,0,1.0,False,False,False,0,1,False,2,1,1,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,1,1,1,5,b3,0,@,b3
10,647,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,11,0,0,1,0,30-40,2 (6),1,0,NaN,2,1,CR,6f37b3b3f3s1w#,NaN,NaN,0,NaN,1,NaN,False,False,False,1,0,False,2,1,5,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,1,1,0,6,f37 b3 b3 f3 s1,2,#,f37
11,648,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,12,0,0,1,0,40-40,2 (7),1,0,NaN,2,1,CR,4f3w#,NaN,NaN,0,NaN,1,NaN,False,False,False,1,0,False,2,1,1,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,1,1,0,4,f3,0,#,f3
12,649,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,13,0,0,1,0,AD-40,2 (8),1,0,NaN,2,1,CR,4b38f3*,NaN,NaN,0,NaN,1,NaN,False,False,True,0,0,False,2,1,3,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,1,1,0,4,b38 f3,1,*,b38


## Geração das features

In [414]:
features = ['Surface',
            'Round',
            'Grand Slam',
            'is_second_service',
            #'dir_srv',
            #'rallyCount',
            #'rally_tratada',
            'isUnforced',
            'isForced',
            "serve_return",
            "how_ended"
            ]
X = df[features]

y = df[['dir_srv','rally_tratada']]

In [415]:
feat_categorical = ['Surface','Round','serve_return','how_ended']

## separação - treino, teste e validação

In [418]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_encoded = pd.DataFrame(encoder.fit_transform(X[feat_categorical]))
X_encoded.columns = encoder.get_feature_names_out(feat_categorical)



In [419]:
X_encoded.reset_index(drop=True, inplace=True)  # Reset index to match X
X.reset_index(drop=True, inplace=True)  # Reset index to match X_encoded
X = pd.concat([X, X_encoded], axis=1)
X = X.drop(categorical_cols, axis=1)

In [420]:
# Train-Val Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



print(len(X_train), 'train examples')
#print(len(X_val), 'validation examples')
print(len(X_test), 'test examples')

3535 train examples
1515 test examples


## Definição do modelo

In [421]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

xgb_model = MultiOutputRegressor(XGBRegressor(objective="reg:squarederror"))
xgb_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...))

In [422]:
# Instanciate model
#from xgboost import XGBRegressor
#xgb_model = XGBRegressor(objective="multi:softmax", num_class=3)  # Assuming 3 serve directions (4, 5, 6)
#xgb_model.fit(X_train, y_train)

## Resultado final / evaluate

In [423]:
y_pred = xgb_model.predict(X_test)

In [424]:
from sklearn.metrics import mean_squared_error
for i in range(2):  # Two outputs: dir_srv and rally_tratada
    y_true = y_test.iloc[:, i]
    y_pred_single = y_pred[:, i]
    mse = mean_squared_error(y_true, y_pred_single)
    rmse = mse**0.5
    print(f"Root Mean Squared Error (RMSE) for Output {i+1}: {rmse}")

Root Mean Squared Error (RMSE) for Output 1: 0.9152523872148007
Root Mean Squared Error (RMSE) for Output 2: 1.9999031790569135


In [425]:
# Make predictions for the example input X
predictions = xgb_model.predict(X)

# Print the predictions for each output
print("Predicted dir_srv:", predictions[:, 0])
print("Predicted rally_tratada:", predictions[:, 1])

Predicted dir_srv: [4.3270025 4.816666  5.140215  ... 4.5019937 4.3270025 4.8612876]
Predicted rally_tratada: [2.7126822  0.14566167 2.2446556  ... 0.2656335  2.7126822  0.07274437]


In [395]:
# Example input X for prediction
example_X = pd.DataFrame({
    "Surface": ["clay"],
    "Round": ["QuarterFinal"],
    "Grand Slam": [0],
    "is_second_service": [1],
    "dir_srv": [5],
    "rally_tratada": [0],
    "isUnforced": [0],
    "isForced": [0],
    "serve_return": ["forehand"],
    "how_ended": ["winner"],
    "i_win": [1],
    "i_serve": [1]
})

In [396]:

example_X_encoded = pd.DataFrame(encoder.transform(example_X[categorical_cols]))
example_X_encoded.columns = encoder.get_feature_names_out(categorical_cols)
example_X_encoded.reset_index(drop=True, inplace=True)  # Reset index to match example_X
example_X.reset_index(drop=True, inplace=True)  # Reset index to match example_X_encoded
example_X = pd.concat([example_X, example_X_encoded], axis=1)


In [397]:
example_X.shape

(1, 16)

In [426]:
# Make predictions for the example input X
predictions = xgb_model.predict(example_X)

# Print the predictions for each output
print("Predicted dir_srv:", predictions[:, 0])
print("Predicted rally_tratada:", predictions[:, 1])

ValueError: Feature shape mismatch, expected: 86, got 16

(5050, 82)